# The Black-Scholes PDE

https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/2.1%20Black-Scholes%20PDE%20and%20sparse%20matrices.ipynb

In this notebook let's try solving the famous Black-Scholes (BS) Partial Differential Equation (PDE) also know as heat equation :
\begin{equation}
	\frac{\partial \mathrm V}{ \partial \mathrm t } + \frac{1}{2}\sigma^{2} \mathrm S^{2} \frac{\partial^{2} \mathrm V}{\partial \mathrm S^2}
	+ \mathrm r \mathrm S \frac{\partial \mathrm V}{\partial \mathrm S} - \mathrm r \mathrm V= 0
\end{equation}

This time we won't use probability (see notebook BS_ClosedForm) but we will use a fully implicit method, because it is an efficient and stable method. The illustrated approach can be used to solve more complex problems, such as the pricing of different types of derivatives (barrier options, digital options, American options, etc) with small modifications.

In [62]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,6)
import matplotlib as mpl
mpl.style.use('seaborn-whitegrid')
from scipy import sparse
from scipy.sparse.linalg import splu
from scipy.sparse.linalg import spsolve

from IPython.display import display
import sympy; sympy.init_printing()

def display_matrix(m):
    display(sympy.Matrix(m))

## PDE in log-variables

Let us consider the geometric Brownian motion (GBM), described by the SDE:

$$\frac{dS_t}{S_t} = \mu dt + \sigma dW_t$$

Let $V(t,S_t)$ be the value of a European call option.

By the martingale pricing theory, in an arbitrage-free market there exists an equivalent martingale measure $\mathbb{Q}$ such that the discounted option price is a $\mathbb{Q}$-martingale. (Remember that under $\mathbb{Q}$ the drift $\mu$ is replaced by the risk free drift $r$.)

In log-variables the BS PDE is:

\begin{equation}
	\frac{\partial \mathrm V(t,x)}{ \partial \mathrm t } +( r - \frac{1}{2}\sigma^{2} ) \frac{\partial \mathrm V(t,x)}{\partial x} + \frac{1}{2}\sigma^{2} \frac{\partial^{2} \mathrm V (t,x)}{\partial x^2}
 - \mathrm r \mathrm V(t,x)= 0
\end{equation}

For an option with strike $K$ and maturity $T$, the boundary conditions are:

CALL:
*   Terminal : $$ V(t,x) = max(e^x-K,0)$$
*   Lateral $$ \lim_{x\to-\infty} V(t,x) = 0  \text{    and} \lim_{x\to+\infty} V(t,x) \sim e^x -Ke^{-r(T-t)}$$

Put:
*   Terminal : $$ V(t,x) = max(K-e^x,0)$$
*   Lateral $$ \lim_{x\to-\infty} V(t,x) \sim Ke^{-r(T-t)}  \text{    and} \lim_{x\to+\infty} V(t,x) = 0$$




## Derivative approximation

Finite difference methods are a technique for obtaining numerical solutions of PDEs. The idea underlying finite-difference methods is to replace the partial derivatives occurring in the PDE by finite difference approximations. If we assume that $V$ is a smooth function, we can use the Taylor series expansion near the point of interest. For a $\Delta t > 0$ we can write :

$$ V(t+\Delta t, x) \approx V(t,x) + \frac{\partial \mathrm V(t,x)}{ \partial \mathrm t }Δt +  \frac{1}{2}\sigma^{2} \frac{\partial^{2} \mathrm V (t,x)}{\partial t^2}Δt^2 + $$
$$ V(t-\Delta t, x) \approx V(t,x) - \frac{\partial \mathrm V(t,x)}{ \partial \mathrm t }Δt +  \frac{1}{2}\sigma^{2} \frac{\partial^{2} \mathrm V (t,x)}{\partial t^2}Δt^2 +$$

An analogous approximation can be done for $V(t,x+\Delta x)$ with $\Delta x > 0$.

If we want to approximate the partial derivative with respect to time, we obtain the following finite difference approximation :

\begin{equation}
	\frac{\partial \mathrm V(t,x)}{ \partial \mathrm t }  = \frac{V(t+Δt,x) - V(t,x)}{Δt}
\end{equation}

Also called **forward difference**, since the differencing is in the forward $t$ direction. We can also consider the **backward difference** :

\begin{equation}
	\frac{\partial \mathrm V(t,x)}{ \partial \mathrm t }  = \frac{V(t,x) - V(t-Δt,x)}{Δt}
\end{equation}

and the **central difference**:

\begin{equation}
	\frac{\partial \mathrm V(t,x)}{ \partial \mathrm t }  = \frac{V(t+Δt,x) - V(t,x-Δt)}{2Δt}
\end{equation}


If we want to approximate the first order partial derivative with respect to $x$, we obtain the following finite **central difference** approximation :

\begin{equation}
	\frac{\partial \mathrm V(t,x)}{ \partial x }  = \frac{V(t,x+Δx) - V(t,x - Δx)}{2Δx}
\end{equation}

For second order derivatives, such as $\frac{\partial^2 \mathrm V(t,x)}{ \partial x^2 }$, we can use the symmetric central difference approximation for a $\Delta x > 0$:

\begin{equation}
	\frac{\partial^2 \mathrm V(t,x)}{ \partial x^2 }  = \frac{V(t,x+Δx) - V(t,x - Δx) - 2V(t,x)}{Δx^2}
\end{equation}

The use of the **forward** and **backward difference** approximation leads to the **explicit** and **implicit** finite difference schemes respectively. The **central difference** is not used for the time variable because it leads to bad numerical schemes. But it is common to use it for the space variable.

## Implicit discretization

First we have to restrict the theoretical infinite domain to the finite region $[t_0,T]\, \times \, [A_1,A_2]$, with $A_1 < A_2$.

The next step is to replace $[t_0,T]\times [A_1,A_2]$ by a discrete grid:

For $n = 0,1, ... N \in \mathbb{N}$, define the discrete time step $ \Delta t = \frac{T - t_0}{N} $ such that $t_n = t_0 + n \Delta t$. For $i = 0,1, ... M \in \mathbb{N}$, define the discrete space step $ \Delta x = \frac{A_2 - A_1}{M} $ such that $x_i = A_1 + i \Delta x$.

The grid is divided into equally spaced nodes of distance $\Delta x$ in the x-axis, and of distance $\Delta t$ in the t-axis.

The mesh points have the form $(t_0 + n \Delta t, A_1 + i \Delta x)$. At this point we concern ourselves only with the values of $V(t,x)$ on the mesh nodes. We call :  

$$V(t_0 + nΔt, A1 + iΔx) =  V_i^n$$

We apply the backward discretization (implicit scheme) for the time derivative, and a central discretization for the first order space derivative.

We are interested in the value of V at time $t_0$. We know the values $V^N$ corresponding to the terminal conditions. The algorithm consists in finding the values $V^n$ given the knowledge of the values $V^{n+1}$.

The discretized equation becomes :

$$ \frac{V_i^{n+1} -V_i^{n}}{Δt} + (r - \frac{1}{2}σ^2)\frac{V_{i+1}^{n} -V_{i-1}^{n}}{2Δx} + \frac{1}{2}σ^2\frac{V_{i+1}^{n} -V_{i-1}^{n} - 2V_{i}^{n}}{Δx^2} - r V_i^n = 0$$

Rearranging the terms:

\begin{align*}
  V_i^{n+1} &= V_i^n(1+rΔt + σ^2 \frac{Δt}{Δx^2}) \\ 
  &+ V_{i+1}^n(-(r-\frac{1}{2}σ^2)\frac{Δt}{2Δx}- \frac{1}{2}σ^2\frac{Δt}{Δx^2} \\
  &+ V_{i-1}^n((r-\frac{1}{2}σ^2)\frac{Δt}{2Δx} - \frac{1}{2}σ^2\frac{Δt}{Δx^2}
\end{align*}

We can rename the coefficients such that:

$$V_i^{n+1} = \mathrm a V_{i-1}^{n} + \mathrm b V_{1}^{n} + \mathrm c V_{1+1}^{n}$$

and write it in matrix form:

$$\begin{pmatrix} V_{1}^{n+1} \\
  V_{2}^{n+1} \\
  \vdots  \\
  V_{M-2}^{n+1}  \\
  V_{M-1}^{n+1}
  \end{pmatrix} =
\begin{pmatrix} b & c & 0 & \dots & 0\\
  a & b & c & 0 & 0 \\
  0 & \ddots & \ddots & \ddots & 0 \\
  \vdots & 0 & a & b & c \\
  0 & 0 & 0 & a & b 
  \end{pmatrix} .
  \begin{pmatrix} V_{1}^{n} \\
  V_{2}^{n} \\
  \vdots  \\
  V_{M-2}^{n}  \\
  V_{M-1}^{n}
  \end{pmatrix}
  +
  \begin{pmatrix} aV_{0}^{n} \\
  0 \\
  \vdots  \\
  0  \\
  cV_{M}^{n}
  \end{pmatrix}
   $$

The system

$$ V^{n+1} = \mathcal{D}V^n + B$$

can be solved easily for $V^{n}$ by inverting the matrix $\mathcal{D}$.



## Numerical solution of the PDE

Let's now solve the vector equation derived in the section "Implicit discretization".

In this case we consider a call option with strike $K$, maturity $T$. The stock price $S_0$ is not relevant for the algorithm. We will use it in the end to compute the value of the option for $S_0$.

A common practice is to choose the computational region between $3K$ and $\frac{K}{3}$. Then we have $A_1 = \log \frac{K}{3}$ and $A_2 = \log 3K$.

The values of the parameter are:

In [53]:
#Option parameters
r = 0.01
sigma = 0.2                
S0 = 100
X0 = np.log(S0)          
K = 101
tau = 2

### Call option valuation

For a put option with strike $K$ and maturity $T$, the boundary conditions are:

*   Terminal : $$ V(t,x) = max(e^x-K,0)$$
*   Lateral $$ \lim_{x\to-\infty} V(t,x) = 0  \text{    and} \lim_{x\to+\infty} V(t,x) \sim e^x -Ke^{-r(T-t)}$$

In [54]:
#Grid parameters
Nspace = 3000   # M space steps
Ntime = 2000    # N time steps   

S_max = 3*float(K)                
S_min = float(K)/3

x_max = np.log(S_max)  # A2
x_min = np.log(S_min)  # A1

x, dx = np.linspace(x_min, x_max, Nspace, retstep=True)   # space discretization
T, dt = np.linspace(0, tau, Ntime, retstep=True)       # time discretization
Payoff = np.maximum(np.exp(x)-K,0)          # Call payoff

V = np.zeros((Nspace,Ntime))       # grid initialization
offset = np.zeros(Nspace-2)        # vector to be used for the boundary terms   

# Conditions
V[:,-1] = Payoff                   # terminal conditions 
V[-1,:] = np.exp(x_max) - K * np.exp(-r* T[::-1] )  # boundary/lateral condition (maximum profit)
V[0,:] = 0                         # boundary/lateral condition (lowest profit)

In [55]:
# construction of the tri-diagonal matrix D
sig2 = sigma*sigma; dxx = dx * dx

a = ( (dt/2) * ( (r-0.5*sig2)/dx - sig2/dxx ) )
b = ( 1 + dt * ( sig2/dxx + r ) )
c = (-(dt/2) * ( (r-0.5*sig2)/dx + sig2/dxx ) )

D = sparse.diags([a, b, c], [-1, 0, 1], shape=(Nspace-2, Nspace-2)).tocsc()

In [56]:
# Backward iteration
import time
sTime = time.time()
for i in range(Ntime-2,-1,-1):
    offset[0] = a * V[0,i]
    offset[-1] = c * V[-1,i]; 
    V[1:-1,i] = spsolve( D, (V[1:-1,i+1] - offset))

eTime = time.time() - sTime 

Now that we have grid of option prices depending on the stock price at time $t_0$, we can now find/interpolate the option price for $S_0 = 100$

https://numpy.org/doc/stable/reference/generated/numpy.interp.html

In [57]:
# finds the option at S0
call = np.interp(X0, x, V[:,0])
print("Call Option Price is : %f in %f secondes" %(call, eTime))

Call Option Price is : 11.696317 in 4.529993 secondes


### Put option valuation

For a put option with strike $K$ and maturity $T$, the boundary conditions are:

*   Terminal : $$ V(t,x) = max(K-e^x,0)$$
*   Lateral $$ \lim_{x\to-\infty} V(t,x) \sim Ke^{-r(T-t)}  \text{    and} \lim_{x\to+\infty} V(t,x) = 0$$

In [58]:
#Grid parameters
Nspace = 3000   # M space steps
Ntime = 2000    # N time steps   

S_max = 3*float(K)                
S_min = float(K)/3

x_max = np.log(S_max)  # A2
x_min = np.log(S_min)  # A1

x, dx = np.linspace(x_min, x_max, Nspace, retstep=True)   # space discretization
T, dt = np.linspace(0, tau, Ntime, retstep=True)       # time discretization
Payoff = np.maximum(K - np.exp(x),0)          # Call payoff

V = np.zeros((Nspace,Ntime))       # grid initialization
offset = np.zeros(Nspace-2)        # vector to be used for the boundary terms   

# Conditions
V[:,-1] = Payoff                   # terminal conditions 
V[-1,:] = K * np.exp(-r* T[::-1] )  # boundary/lateral condition (maximum profit)
V[0,:] = 0                         # boundary/lateral condition (lowest profit)

In [59]:
# construction of the tri-diagonal matrix D
sig2 = sigma*sigma; dxx = dx * dx

a = ( (dt/2) * ( (r-0.5*sig2)/dx - sig2/dxx ) )
b = ( 1 + dt * ( sig2/dxx + r ) )
c = (-(dt/2) * ( (r-0.5*sig2)/dx + sig2/dxx ) )

D = sparse.diags([a, b, c], [-1, 0, 1], shape=(Nspace-2, Nspace-2)).tocsc()

# Backward iteration
sTime = time.time()
for i in range(Ntime-2,-1,-1):
    offset[0] = a * V[0,i]
    offset[-1] = c * V[-1,i]; 
    V[1:-1,i] = spsolve( D, (V[1:-1,i+1] - offset))

eTime = time.time()- sTime

In [61]:
# finds the put option at S0
put = np.interp(X0, x, V[:,0])
print("Put Option Price is : %f in %f secondes" %(put, eTime))

Put Option Price is : 10.692761 in 4.445136 secondes
